In [1]:
import time
import re
import pandas as pd
import numpy as np
import requests
import pickle
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import csv
from tqdm import tqdm
import datetime

# 인기영상 크롤링
- 주제에 관계없이 가장 많은 리뷰를 뽑아쓸 수 있는 방법

In [2]:
def get_trending_video():
    url = 'https://www.youtube.com/feed/trending'

    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5)
    
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height

    
    load_box = driver.find_elements_by_css_selector('#grid-container ytd-video-renderer')
    
    info_dict = {'title' : [],
                'link' : [],
                'Youtuber' : [],
                'view' : [],
                'date' : []}
    for video in load_box:
        try: 
            #제목
            title = video.find_element_by_css_selector('#video-title').text
            #url
            link = video.find_element_by_css_selector('#video-title').get_attribute('href')
            #유튜버
            Youtuber = video.find_element_by_css_selector('#text a').text
            #조회수
            view = video.find_element_by_css_selector('#metadata-line span:nth-child(1)').text
            #날짜
            date = video.find_element_by_css_selector('#metadata-line span:nth-child(2)').text

            info_dict['title'].append(title)
            info_dict['link'].append(link)
            info_dict['Youtuber'].append(Youtuber)
            info_dict['view'].append(view)
            info_dict['date'].append(date)
        except:
            pass

    df = pd.DataFrame(info_dict)
    
    return df

In [3]:
df = get_trending_video()

저장할때 날짜정보 저장해서, 하루 전 데이터를 불러오고 url비교해서 크롤링할 url이랑 아닌거랑 구분

입력어 받는 파이썬파일로.  
원하는리뷰수를 인풋?  
-이거계산하는거만들어보고  
url도 인풋으로

In [74]:
#스크롤 횟수

def Youtube_crawl(url, scroll_cnt):
    scroll_cnt = scroll_cnt

    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(2) #댓글창로딩

    body = driver.find_element_by_tag_name('body')

    while scroll_cnt:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        scroll_cnt -= 1

    reviews = driver.find_elements_by_xpath('//*[@id="main"]')
    likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
    print(scroll_cnt, len(reviews))


    if len(reviews) != len(likes):
        reviews = reviews[:-1] #마지막 review에 Youtube Premium 광고가 섞이는경우가있음

    df = pd.DataFrame(columns=['리뷰', '좋아요'])

    review_list = []
    like_list = []
    for review, like in zip(reviews, likes):
        review_list.append(review.text)
        like_list.append(like.text)

    df['리뷰'] = review_list
    df['좋아요'] = like_list

### page_down을 사용하면 버벅거리기에 느리다.
- page_down 대신에 scrollHeight 이용

In [ ]:
def Youtube_crawl_2(url):
    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5) #댓글창로딩

    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    #댓글 최하단까지 이동
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height
    
    #자세히 보기를 클릭해 리뷰 잘림 현상 방지
    load_more = driver.find_elements_by_xpath('//*[@id="more"]/span')

    for load in load_more:
        if load.text == '자세히 보기':
            load.click()

    reviews = driver.find_elements_by_xpath('//*[@id="main"]')
    likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')

    if len(reviews) != len(likes):
        reviews = reviews[:-1] #마지막 review에 Youtube Premium 광고가 섞이는경우가있음

    df = pd.DataFrame(columns=['리뷰', '좋아요'])

    review_list = []
    like_list = []
    for review, like in zip(reviews, likes):
        review_list.append(review.text)
        like_list.append(like.text)

    df['리뷰'] = review_list
    df['좋아요'] = like_list
    
    return df

### 답글이 있는 댓글을 못가져온다

In [2]:
def Youtube_crawl_3(url):
    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5) #댓글창로딩

    #모든 댓글 불러오기
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height
    
    #대댓글 불러오기
    load_reply = driver.find_elements_by_css_selector('#contents #text')

    for load in load_reply:
        #답글을 남기는 '답글' 과 '답글 n개 보기' 중 'n개 보기' 필터링
        if '보기' in load.text: 
            driver.execute_script('arguments[0].click();',load)
            #load.click()의 경우 오류가 발생
        else:
            pass
    
    #대댓글 더 불러오기
    while True:
        try:
            check_point = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

            if len(check_point):
                #답글이 일정 수준을 넘기면 '답글 더보기'가 등장, 이를 클릭하기 위함
                load_more_reply = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

                for load in load_more_reply:
                    driver.execute_script('arguments[0].click();',load)
        except:
            break

    
    #답글에도 자세히보기를 사용해야할경우가있기때문에 자세히보기 클릭은 답글을 다 연 다음에
    load_detail = driver.find_elements_by_xpath('//*[@id="more"]/span')

    for load in load_detail:
        if load.text == '자세히 보기':
            driver.execute_script('arguments[0].click();',load)
        else:
            pass

    reviews = driver.find_elements_by_css_selector('#content-text')
    likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')

    if len(reviews) != len(likes):
        reviews = reviews[:-1] #마지막 review에 Youtube Premium 광고가 섞이는경우가있음

    df = pd.DataFrame(columns=['리뷰', '좋아요'])

    review_list = []
    like_list = []
    for review, like in zip(reviews, likes):
        review_list.append(review.text)
        like_list.append(like.text)

    df['리뷰'] = review_list
    df['좋아요'] = like_list
    
    #좋아요 전처리
    df['좋아요'] = df['좋아요'].apply(lambda x: re.sub('천 | 만','000',x)) #천의 경우는 1천 / 만의 경우는 1.1만 의 경우로 표현됨
    
    #좋아요가 없는 칼럼은 1로 대체
    indexes = df[df['좋아요'] == ''].index
    df.loc[indexes, '좋아요'] = 1
    
    return df

Youtube_crawl_3의 경우 리뷰와 좋아요를 따로 뽑고 하나로 합치는 방식이기에, 좋아요와 리뷰의 순서가 맞지 않을 가능성 발생
따라서 각 리뷰가 담긴 박스 전체를 추출하고, 그 안에서 ID, 리뷰, 좋아요, 날짜를 추출하는 방식으로 변경

In [4]:
def Youtube_crawl_4(url):
    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5) #댓글창로딩
    
    start = time.time()

    #모든 댓글 불러오기
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height

    #대댓글 불러오기
    load_reply = driver.find_elements_by_css_selector('#contents #text')

    for load in load_reply:
        #답글을 남기는 '답글' 과 '답글 n개 보기' 중 'n개 보기' 필터링
        if '보기' in load.text: 
            driver.execute_script('arguments[0].click();',load)
            #load.click()의 경우 오류가 발생
        else:
            pass

    #대댓글 더 불러오기
    while True:
        try:
            check_point = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

            if len(check_point):
                #답글이 일정 수준을 넘기면 '답글 더보기'가 등장, 이를 클릭하기 위함
                load_more_reply = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

                for load in load_more_reply:
                    driver.execute_script('arguments[0].click();',load)
        except:
            break
    
    load_content = driver.find_elements_by_xpath('//*[@id="main"]')

    content_info = {
        'author':[],
       'date' : [],
       'review' : [],
       'like' : []
    }
  
    for content in load_content:
        try: #Youtube Premium이 걸릴경우가 있다
            #ID
            author = content.find_element_by_css_selector('#author-text span').text
            #날짜
            date = content.find_element_by_css_selector('#header-author  yt-formatted-string  a').text
            #리뷰
            review = content.find_element_by_css_selector('#content-text').text
            #like
            like = content.find_element_by_css_selector('#vote-count-middle').text

            content_info['author'].append(author)
            content_info['date'].append(date)
            content_info['review'].append(review)
            content_info['like'].append(like)
        except:
            pass

    df = pd.DataFrame(content_info)

    #like 전처리
    df['like'] = df['like'].apply(lambda x: re.sub('천 | 만','000',x)) #천의 경우는 1천 / 만의 경우는 1.1만 의 경우로 표현됨

    #like가 없는 칼럼은 1로 대체
    indexes = df[df['like'] == ''].index
    df.loc[indexes, 'like'] = 1
    
    end = time.time()
    
    print('time: {}'.format(end-start))
    
    return df

# 최종본

In [36]:
def YoutubeCrawler(df):
    
    #시작 url 설정 - 사실 어느 사이트건 상관은 없다.
    start_url = 'https://www.youtube.com/'
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(start_url)
    
    link_list = df['link']
    link_length = len(link_list)
    
    for j in tqdm(range(link_length)):
#         try:
            print('{}번째 동영상'.format(j))

            #필요한 url 새창 띄우기
            driver.execute_script("window.open('{},_blank');".format(link_list[j+3]))
            driver.switch_to.window(driver.window_handles[-1]) #가장 최근에 연 창 조절

            time.sleep(5) #댓글창로딩

            start = time.time()

            #모든 댓글 불러오기
            last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

            while True:
                driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
                time.sleep(7)
                new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
                
                if new_page_height == last_page_height:
                    break
                else:
                    last_page_height = new_page_height

            check_point_1 = time.time()
            print('마지막까지 내리기 : {}'.format(check_point_1 - start))

            #대댓글 불러오기
            load_reply = driver.find_elements_by_css_selector('#contents #text')

            for load in load_reply:
                #답글을 남기는 '답글' 과 '답글 n개 보기' 중 'n개 보기' 필터링
                if '보기' in load.text: 
                    driver.execute_script('arguments[0].click();',load)
                    #load.click()의 경우 오류가 발생
                else:
                    pass

            check_point_2 = time.time()
            print('대댓글 불러오기 (1) : {}'.format(check_point_2 - check_point_1))

            #대댓글 더 불러오기
            while True:
                check_point = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

                if len(check_point):
                    #답글이 일정 수준을 넘기면 '답글 더보기'가 등장, 이를 클릭하기 위함
                    load_more_reply = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

                    for load in load_more_reply:
                        try:
                            driver.execute_script('arguments[0].click();',load)
                        except:
                            break
                else:
                    break

            check_point_3 = time.time()
            print('대댓글 불러오기 (2) : {}'.format(check_point_3 - check_point_2))

             #답글에도 자세히보기를 사용해야할경우가있기때문에 자세히보기 클릭은 답글을 다 연 다음에
            load_detail = driver.find_elements_by_xpath('//*[@id="more"]/span')

            for load in load_detail:
                if load.text == '자세히 보기':
                    driver.execute_script('arguments[0].click();',load)
                else:
                    pass

            check_point_4 = time.time()
            print('자세히 보기 : {}'.format(check_point_4 - check_point_3))
            
            load_content = driver.find_elements_by_xpath('//*[@id="main"]')
            print('총 리뷰: {}'.format(len(load_content)))
            
            content_info = {
                'author':[],
            #    'date' : [],
               'review' : [],
               'like' : []
            }

            content_length = len(load_content)
            print('총 리뷰: {}'.format(content_length))
            for i in tqdm(range(content_length)):
                try: #Youtube Premium이 걸릴경우가 있다
                    #ID
                    author = load_content[i].find_element_by_css_selector('#author-text span').text
                    content_info['author'].append(author)
                except:
                    content_info['author'].append('None')

            #     try:    
            #         #날짜
            #         date = load_content[i].find_element_by_css_selector('#header-author  yt-formatted-string  a').text
            #         content_info['date'].append(date)
            #     except:
            #         content_info['date'].append('None')

                try:    
                    #리뷰
                    review = load_content[i].find_element_by_css_selector('#content-text').text
                    content_info['review'].append(review)
                except:
                    content_info['review'].append('None')
                try:
                    #like
                    like = load_content[i].find_element_by_css_selector('#vote-count-middle').text
                    content_info['like'].append(like)
                except:
                    content_info['like'].append('None')


            now = datetime.datetime.now()
            now = now.strftime('%Y%m%d')
            df = pd.DataFrame(content_info)
            df.to_pickle('TrendingVideo_{}_{}.pkl'.format(now, j))
            
            end = time.time()
            print('댓글 크롤링 : {}'.format(end - check_point_4))
            print('처음-끝 : {}'.format(end - start))
            print('{}번째 동영상 끝'.format(j))
            print('--------------------------------')
            
            #대책없이 새창 열고 이전 창에 대한 관리를 안하면 메모리 오류가 나면서 죽어버린다
            driver.close() # 기존 창 닫고
            driver.switch_to.window(driver.window_handles[0]) #첫번째 창으로 이동 & 연결


In [ ]:
 YoutubeCrawler(df)

  0%|                                                                                           | 0/75 [00:00<?, ?it/s]

0번째 동영상
마지막까지 내리기 : 23.0904221534729
대댓글 불러오기 (1) : 1.7884106636047363
대댓글 불러오기 (2) : 6.001567125320435



  1%|▉                                                                                 | 2/175 [00:00<00:12, 13.60it/s]

자세히 보기 : 1.555022954940796
총 리뷰: 175
총 리뷰: 175



 79%|███████████████████████████████████████████████████████████████▌                | 139/175 [00:07<00:02, 17.42it/s]


...

 99%|█████████████████████████████████████████████████████████████████████████████▎| 1914/1930 [12:49<00:07,  2.20it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 1930/1930 [12:55<00:00,  2.49it/s]


댓글 크롤링 : 777.0809533596039
처음-끝 : 1715.138275384903
2번째 동영상 끝
--------------------------------


  4%|███▏                                                                           | 3/75 [54:46<17:10:07, 858.44s/it]

3번째 동영상
마지막까지 내리기 : 479.5038511753082


In [15]:
def Crawl_only_review(df):
    
    #시작 url 설정 - 사실 어느 사이트건 상관은 없다.
    start_url = 'https://www.youtube.com/'
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(start_url)
    
    for i in tqdm(range(len(df[33:]))):
#         try:
            print('{}번째 동영상'.format(i))

            #필요한 url 새창 띄우기
            driver.execute_script("window.open('{},_blank');".format(df['link'][i + 33]))
            driver.switch_to.window(driver.window_handles[-1]) #가장 최근에 연 창 조절

            time.sleep(5) #댓글창로딩

            start = time.time()

            #모든 댓글 불러오기
            last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

            while True:
                driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
                time.sleep(7)
                new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
                
                if new_page_height == last_page_height:
                    break
                else:
                    last_page_height = new_page_height

            check_point_1 = time.time()
            print('마지막까지 내리기 : {}'.format(check_point_1 - start))

            #대댓글 불러오기
            load_reply = driver.find_elements_by_css_selector('#contents #text')

            for load in load_reply:
                #답글을 남기는 '답글' 과 '답글 n개 보기' 중 'n개 보기' 필터링
                if '보기' in load.text: 
                    driver.execute_script('arguments[0].click();',load)
                    #load.click()의 경우 오류가 발생
                else:
                    pass

            check_point_2 = time.time()
            print('대댓글 불러오기 (1) : {}'.format(check_point_2 - check_point_1))

            #대댓글 더 불러오기
            while True:
                check_point = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

                if len(check_point):
                    #답글이 일정 수준을 넘기면 '답글 더보기'가 등장, 이를 클릭하기 위함
                    load_more_reply = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

                    for load in load_more_reply:
                        try:
                            driver.execute_script('arguments[0].click();',load)
                        except:
                            break
                else:
                    break

            check_point_3 = time.time()
            print('대댓글 불러오기 (2) : {}'.format(check_point_3 - check_point_2))

             #답글에도 자세히보기를 사용해야할경우가있기때문에 자세히보기 클릭은 답글을 다 연 다음에
            load_detail = driver.find_elements_by_xpath('//*[@id="more"]/span')

            for load in load_detail:
                if load.text == '자세히 보기':
                    driver.execute_script('arguments[0].click();',load)
                else:
                    pass

            check_point_4 = time.time()
            print('자세히 보기 : {}'.format(check_point_4 - check_point_3))

            reviews = driver.find_elements_by_css_selector('#content-text')
            print('총 리뷰: {}'.format(len(reviews)))

            review_list = [r.text for r in reviews]

            end = time.time()
            print('댓글 크롤링 : {}'.format(end - check_point_4))
            print('처음-끝 : {}'.format(end - start))
            
            temp_df = pd.DataFrame(review_list, columns = ['review'])
            temp_df.to_pickle('trending_video_{}.pkl'.format(i + 20))
            
            #대책없이 새창 열고 이전 창에 대한 관리를 안하면 메모리 오류가 나면서 죽어버린다
            driver.close() # 기존 창 닫고
            first_tab = driver.window_handles[0] 
            driver.switch_to.window(window_name=first_tab) #첫번째 창으로 이동 & 연결

#         except:
#             print('{}번째 동영상에서 에러 발생'.format(i))

# 대댓글 불러오기에서 오류가 걸린다

In [176]:
def crawl_review(urls):
    
    '''
    기존 코드
    -------------------------------------------------------------------
    f = open('유튜브 리뷰_3.csv', 'w', encoding='utf-8-sig', newline='')
    f.close()
    
    for i in tqdm(range(len(df))):
        title = df['title'][i]
        print(title)
        
        url = df['link'][i]

        #자신의 크롬 버전에 맞는 드라이버 다운
        driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
        driver.get(url)

        time.sleep(10) #댓글창로딩
        
    --------------------------------------------------------------------
    새로운 창을 불러올때마다 수동으로 클릭을 해야지만 정보를 제대로 불러왔다.
    이 문제점을 해결하기 위해 새로운 창을 띄우고 switch하는 방식으로 변경

    '''
    
    #시작 url 설정 - 사실 어느 사이트건 상관은 없다.
    start_url = 'https://www.youtube.com/'
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(start_url)
    
#     f = open(csv_file, 'w', encoding='utf-8', newline='')
#     f.close()

    for i in tqdm(range(len(urls))):    
        driver.execute_script("window.open('{},_blank');".format(urls[i]))
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(10) #댓글창로딩
        start = time.time()

        #모든 댓글 불러오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
            time.sleep(15)
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

            if new_page_height == last_page_height:
                break

            last_page_height = new_page_height

        check_point_1 = time.time()
        print('마지막까지 내리기 : {}'.format(check_point_1 - start))

#         #대댓글 불러오기
#         load_reply = driver.find_elements_by_css_selector('#contents #text')

#         for load in load_reply:
#             #답글을 남기는 '답글' 과 '답글 n개 보기' 중 'n개 보기' 필터링
#             if '보기' in load.text: 
#                 driver.execute_script('arguments[0].click();',load)
#                 #load.click()의 경우 오류가 발생
#             else:
#                 pass

#         check_point_2 = time.time()
#         print('대댓글 불러오기 (1) : {}'.format(check_point_2 - check_point_1))

#         #대댓글 더 불러오기
#         while True:
#             try:
#                 check_point = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

#                 if len(check_point):
#                     #답글이 일정 수준을 넘기면 '답글 더보기'가 등장, 이를 클릭하기 위함
#                     load_more_reply = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

#                     for load in load_more_reply:
#                         driver.execute_script('arguments[0].click();',load)
#             except:
#                 break

#         check_point_3 = time.time()
#         print('대댓글 불러오기 (2) : {}'.format(check_point_3 - check_point_2))

         #답글에도 자세히보기를 사용해야할경우가있기때문에 자세히보기 클릭은 답글을 다 연 다음에
        load_detail = driver.find_elements_by_xpath('//*[@id="more"]/span')

        for load in load_detail:
            if load.text == '자세히 보기':
                driver.execute_script('arguments[0].click();',load)
            else:
                pass

        check_point_4 = time.time()
#         print('자세히 보기 : {}'.format(check_point_4 - check_point_3))

        reviews = driver.find_elements_by_css_selector('#content-text')
        print('총 리뷰: {}'.format(len(reviews)))

        review_list = [r.text for r in reviews]

        end = time.time()
        print('댓글 크롤링 : {}'.format(end - check_point_4))
        print('처음-끝 : {}'.format(end - start))
        
        temp_df = pd.DataFrame(review_list, columns=['review'])
        temp_df.to_pickle('sweet_home_{}.pkl'.format(i))

#         with open(csv_file, 'a', encoding='utf-8', newline='') as f:
#             for review in review_list:
#                 f.write(review)

# 스위트홈 리뷰

In [167]:
urls = ['https://www.youtube.com/watch?v=HhgcXuN0MlU&t=604s',
       'https://www.youtube.com/watch?v=cxO6Q6jiBys',
       'https://www.youtube.com/watch?v=RbGcF_CAOLQ',
       'https://www.youtube.com/watch?v=EqfsrKC5kvE',
       'https://www.youtube.com/watch?v=M738oS60_9g',
       ]

In [168]:
crawl_review(urls)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

마지막까지 내리기 : 32.08852672576904
총 리뷰: 40


 20%|████████████████▊                                                                   | 1/5 [00:42<02:51, 42.98s/it]

댓글 크롤링 : 0.4324047565460205
처음-끝 : 32.84386658668518
마지막까지 내리기 : 804.188060760498
총 리뷰: 1580


 40%|█████████████████████████████████▏                                                 | 2/5 [17:41<16:47, 335.67s/it]

...

댓글 크롤링 : 2.3932175636291504
처음-끝 : 126.80533909797668
마지막까지 내리기 : 698.6648700237274
총 리뷰: 1354


 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [32:50<07:04, 424.31s/it]

댓글 크롤링 : 29.98800253868103
처음-끝 : 754.9723989963531


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [33:13<00:00, 398.61s/it]

마지막까지 내리기 : 12.062984943389893
총 리뷰: 0
댓글 크롤링 : 0.007978677749633789
처음-끝 : 12.081934213638306


# csv 파일 저장문제
- '\n'으로 인해 오류가 발생  
- 하나의 리뷰가 여러개로 분리되거나, 여러개의 리뷰가 하나로 합쳐진다.

In [155]:
write = []
f = open('test.csv', 'w', encoding='utf-8-sig', newline='')
f.close()

In [156]:
df = pd.DataFrame(review_list, columns=['review'] )

In [158]:
with open('test.csv', 'a', encoding='utf-8-sig', newline='') as f:       
    for review in df['review']:
        write.append(review)
        writer = csv.writer(f)
        writer.writerow(write)
        write = []

In [159]:
with open('test.csv', 'r', encoding='utf-8-sig', newline='') as f:
    test = f.readlines()

# CSS selector vs Xpath 속도 비교
- 파싱이 너무 오래걸린다. 둘 사이에 속도 차이가 있는게 아닐까

In [37]:
review_list[0]

'애들아 형 3년째 이러고 있다 내년엔 여친생기겠지 휴....'

In [24]:
start = time.time()
reviews_1= driver.find_elements_by_css_selector('#content-text')
review_list_1 = [r.text for r in reviews_1]
end = time.time()

els = end - start
print(els)

220.78578329086304


In [30]:
start = time.time()
reviews_2 = driver.find_elements_by_xpath('//*[@id="content-text"]')
review_list_2 = [r.text for r in reviews_2]
end = time.time()

els = end - start
print(els)

212.75562000274658


대댓글은 영상의 주제와 관계 없이, 리뷰를 남긴 사람에 집중하는 경우가 많다. ex) 서로를 물어뜯거나, 찐이다 등등  
따라서 해당 주제에 대한 긍정/부정을 파악하는데에 오히려 도움이 되지 않는다고 판단, 대댓글을 불러오지 않는 크롤러

In [102]:
def Youtube_crawl_without_coco(url):
    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5) #댓글창로딩

    #모든 댓글 불러오기
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height
    
    
    #답글에도 자세히보기를 사용해야할경우가있기때문에 자세히보기 클릭은 답글을 다 연 다음에
    load_detail = driver.find_elements_by_xpath('//*[@id="more"]/span')

    for load in load_detail:
        if load.text == '자세히 보기':
            driver.execute_script('arguments[0].click();',load)
        else:
            pass

    load_content = driver.find_elements_by_xpath('//*[@id="main"]')

    content_info = {
#                     'video_title' : [],
#                     'video_view' : [],
#                     'Youtuber': [],
#                     'video_date' : [],
                    'author':[],
                    'date' : [],
                    'review' : [],
                    'like' : []}
    
    for content in load_content:
        #ID
        author = content.find_element_by_css_selector('#author-text span').text
        #날짜
        date = content.find_element_by_css_selector('#header-author  yt-formatted-string  a').text
        #리뷰
        review = content.find_element_by_css_selector('#content-text').text
        #like
        like = content.find_element_by_css_selector('#vote-count-middle').text

        content_info['author'].append(author)
        content_info['date'].append(date)
        content_info['review'].append(review)
        content_info['like'].append(like)

    df = pd.DataFrame(content_info)
    
#         #title
#     df['video_title'] = driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text

#     #view_count
#     df['video_view'] = driver.find_element_by_css_selector('#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer').text

#     #Youtuber
#     df['Youtuber'] = driver.find_element_by_css_selector('#text > a').text

#     #date
#     df['date'] = driver.find_element_by_css_selector('#date > yt-formatted-string').text

#     #like 전처리
#     df['like'] = df['like'].apply(lambda x: re.sub('천 | 만','000',x)) #천의 경우는 1천 / 만의 경우는 1.1만 의 경우로 표현됨

    #like가 없는 칼럼은 1로 대체
    indexes = df[df['like'] == ''].index
    df.loc[indexes, 'like'] = 1
    
    return df

In [101]:
df = Youtube_crawl_without_coco('https://www.youtube.com/watch?v=HhgcXuN0MlU&t=599s')
df.to_pickle('유튜브 리뷰 댓글제거.pkl')

ValueError: arrays must all be same length

보다 많은 리뷰가 성공적으로 불러와졌다


reviews = driver.find_elements_by_xpath('//*[@id="main"]')
reviews = driver.find_elements_by_css_selector('#content-text')

---
xpath로 가져오는 데이터와 css_selector로 가져오는 데이터가 서로 다르다
xpath, css_selector로 가져왔을 때 뭔가 불만족스럽다면 서로를 바꿔보자

### 좋아요 수 전처리
- '1천', or ''

답글과 자세히보기문제

like에 가중치를 둔다면?
댓글 * like로 가중치를두어계산(이때 log2 씌워주고) #like가없다면 차이를두어야겠지?